In [1]:
#importar
#import os
from datetime import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os
#from bs4 import BeautifulSoup
#import re
#from collections import OrderedDict
from selenium.common.exceptions import NoSuchElementException
#import getpass
import pyshorteners

In [ ]:
def realizar_login(url, login1, password1, orgao1):
    """
    Função para realizar login no sistema SEI.

    Args:
        url (str): URL do sistema SEI.
        login1 (str): Nome de usuário para login.
        password1 (str): Senha para login.
        orgao1 (str): Nome do órgão para acesso.

    Returns:
        webdriver: Instância do WebDriver com o usuário autenticado.
    """
    try:
        # Inicializa o navegador
        driver = webdriver.Chrome()
        driver.implicitly_wait(0.5)
        driver.get(url)

        # Localiza os elementos de login
        login = driver.find_element(By.XPATH, '//*[@id="txtUsuario"]')
        password = driver.find_element(By.XPATH, '//*[@id="pwdSenha"]')
        orgao = driver.find_element(By.XPATH, '//*[@id="selOrgao"]')
        submit_button = driver.find_element(By.XPATH, '//*[@id="Acessar"]')

        # Preenche as credenciais
        login.send_keys(login1)
        password.send_keys(password1)
        orgao.send_keys(orgao1)

        # Realiza o login
        submit_button.click()
        time.sleep(3)  # Aguarda carregamento da página após o login

        print("Login realizado com sucesso!")
        return driver
    except Exception as e:
        print(f"Erro ao realizar o login: {e}")
        return None



# Exemplo de uso
url = 'https://sei.economia.gov.br/'
login1 = 'daiana.sales@gestao.gov.br'
password1 = "D!vertidamente081099"
orgao1 = "MGI"

# Realiza login
driver = realizar_login(url, login1, password1, orgao1)

# Se o login foi bem-sucedido, realiza a busca
if driver:
    buscar_arquivos(driver)


searching = driver.find_element(By.XPATH, '//*[@id="infraMenu"]/li[14]/a/span')
searching.click()
time.sleep(1)

#selecionar os pesquisar em documentos
docum_pesq = driver.find_element(By.XPATH, '//*[@id="divOptDocumentos"]/div')
docum_pesq.click()
time.sleep(0.3)

#colocar como tramitação dentro do orgão
chktram = driver.find_element(By.XPATH, '//*[@id="divSinTramitacao"]/div')
chktram.click()
time.sleep(0.2)

# Especifica os termos de pesquisa
espec_pesq = driver.find_element(By.XPATH, '//*[@id="q"]')
espec_pesq.send_keys('indicação ou retificação ou nomear ou nomeação ou ratificar ou retificar ou representante ou exoneração')

# Realizar pesquisa por tipo de processo
tipo_process = driver.find_element(By.XPATH, '//*[@id="selTipoProcedimentoPesquisa"]')
tipo_process.send_keys("Gestão Administrativa: Conselhos, Comissões, Comitês, Grupos de Trabalho e Juntas")
time.sleep(0.5)

# Realiza a pesquisa
b_pesq = driver.find_element(By.XPATH, '//*[@id="sbmPesquisar"]')
b_pesq.click()
time.sleep(0.5)

In [3]:
# Função principal para busca
def buscar_arquivos(driver):
    """
    Realiza a busca de arquivos no sistema SEI após login.

    Args:
        driver (webdriver): Instância do WebDriver autenticada.
    """
    try:
                        
        # Acessa a área de busca
        searching = driver.find_element(By.XPATH, '//*[@id="infraMenu"]/li[14]/a/span')
        searching.click()
        time.sleep(0.5)
        
        #selecionar os pesquisar em documentos
        docum_pesq = driver.find_element(By.XPATH, '//*[@id="divOptDocumentos"]/div')
        docum_pesq.click()
        time.sleep(0.3)
        
        # #Selecionar pesquisar em processos
        # process = driver.find_element(By.XPATH, '//*[@id="divOptProcessos"]/div')
        # process.click()
        # time.sleep(0.2)
        
        # #fazer busca através dos documentos, clicar no botão considerar documentos
        # cons_doc = driver.find_element(By.XPATH, '//*[@id="divSinConsiderarDocumentos"]/div')
        # cons_doc.click()
        # time.sleep(0.5)
        
        #colocar como tramitação dentro do orgão
        chktram = driver.find_element(By.XPATH, '//*[id="chkSinTramitacao"]/div')
        chktram.click()
        time.sleep(0.2)

        # Restringe busca ao órgão específico
        sel_orgao = driver.find_element(By.XPATH, '//*[@id="divSinRestringirOrgao"]/div')
        sel_orgao.click()
        time.sleep(0.5)

        # Especifica os termos de pesquisa
        espec_pesq = driver.find_element(By.XPATH, '//*[@id="q"]')
        espec_pesq.send_keys('indicação ou retificação ou nomear ou nomeação ou ratificar ou retificar ou representante ou exoneração')
        
        # Realizar pesquisa por tipo de processo
        tipo_process = driver.find_element(By.XPATH, '//*[@id="selTipoProcedimentoPesquisa"]')
        tipo_process.send_keys("Gestão Administrativa: Conselhos, Comissões, Comitês, Grupos de Trabalho e Juntas")
        time.sleep(0.5)
        
        # # colocar datas
        data_inicio = driver.find_element('xpath', '//*[@id="txtDataInicio"]')
        data_inicio.send_keys("01/09/2024")
        time.sleep(0.5)
        
        # Realiza a pesquisa
        b_pesq = driver.find_element(By.XPATH, '//*[@id="sbmPesquisar"]')
        b_pesq.click()
        time.sleep(0.5)

        print("Busca realizada com sucesso.\nRestringindo em tipo de processo \n Gestão Administrativa: Conselhos, Comissões, Comitês, Grupos de Trabalho e Juntas \n e dentro do MGI.\n")
    except Exception as e:
        print(f"Erro durante a busca: {e}")

def extrair_dados(driver):     
    """
    Função para realizar web scraping no SEI e retornar os dados em um DataFrame.

    Args:
        driver (webdriver): Instância do Selenium WebDriver.

    Returns:
        pd.DataFrame: DataFrame contendo os dados extraídos.
    """
    def remove_items(lista, item): 
        """Remove todos os itens iguais a `item` de uma lista."""
        return [i for i in lista if i != item]

    # Extraindo os elementos da pesquisa
    tree_elements = driver.find_elements("xpath", '//*[@class="pesquisaTituloEsquerda"]/a')
    # //*[@id="conteudo"]/table/tbody/tr[1]/td[1]/span/text()
    
    list_tree = [element.text for element in tree_elements]
    trees = remove_items(list_tree, '')  # Remover elementos vazios

    # abts = driver.find_elements("xpath", '//*[@class="pesquisaSnippet"]')
    # list_abts = [element.text for element in abts]

    unidades = driver.find_elements("xpath", '//*[@class="pesquisaMetatag"]')
    list_uni = [element.text.split(':') for element in unidades]
    info = [sublist_uni[1] for sublist_uni in list_uni if len(sublist_uni) > 1]  # Removendo listas vazias

    rows = driver.find_elements(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr')
    links = []
    files_name = []
    for i in range(1, len(rows)+1, 3):
        try:
            a = driver.find_element(By.XPATH, f'//*[@id="conteudo"]/table/tbody/tr[{i}]/td[2]/a')
            time.sleep(0.5)
            link = a.get_attribute('href')
            file_name = a.text
            links.append(link)
            files_name.append(file_name)
        except Exception as e:
            print(f"Erro ao processar a linha {i}: {e}")

    # Inicializando o encurtador de links
    shortener = pyshorteners.Shortener(api_key='your_api_key', provider='isgd')
    links_curtos = []
    erros = []
    
    for link in links:
        try:
            # Separando a parte fixa e o hash do link
            base_url, hash_value = link.split('infra_hash=')
            
            # Verificando o tamanho da parte fixa do link antes de encurtar
            if len(base_url) > 1000:  # Ajuste o limite conforme necessário
                raise ValueError(f"Link base muito longo: {base_url}")
            
            # Encurtando a parte fixa do link
            link_curto = shortener.tinyurl.short(base_url)
            time.sleep(0.5)
            
            # Recriando o link com o hash
            link_com_hash = link_curto + 'infra_hash=' + hash_value
            
            links_curtos.append(link_com_hash)
            time.sleep(0.5)
        except Exception as e:
            erros.append((link, str(e)))
            links_curtos.append(None)
    
    # Encontrando o comprimento mínimo entre as listas
    min_length = min(len(trees[::2]), len(trees[1::2]), len(list_abts), len(info[::3]), len(info[1::3]), len(info[2::3]), len(links_curtos))
    
    # Cortando todas as listas para o comprimento mínimo
    dados = {
        "Número do Processo": trees[::2][:min_length],
        "Documento": trees[1::2][:min_length],
        "Resumo": list_abts[:min_length],
        "Unidade": info[::3][:min_length],
        "Usuário": info[1::3][:min_length],
        "Data de Inclusão": info[2::3][:min_length],
        "Links": links_curtos[:min_length]
    }
        
    df = pd.DataFrame(dados)
    
    original_window = driver.current_window_handle 
    
    for index, (link, name) in enumerate(zip(links, files_name)):
        driver.switch_to.new_window('tab')
        driver.get(link)
        with open(f'./colegiado_{name}.html', 'w', encoding='utf-8', errors='ignore') as file:
            file.write(driver.page_source)        
        driver.close()
        driver.switch_to.window(original_window)
    
    return df, links

col = extrair_dados1(driver)

def extrair_dados1(driver):     
    """
    Função para realizar web scraping no SEI e retornar os dados em um DataFrame.

    Args:
        driver (webdriver): Instância do Selenium WebDriver.

    Returns:
        pd.DataFrame: DataFrame contendo os dados extraídos.
        
    """
    def remove_items(lista, item): 
        """Remove todos os itens iguais a `item` de uma lista.""" 
        return [i for i in lista if i != item]
   
    # Extraindo os elementos da pesquisa
    tree_elements = driver.find_elements(By.XPATH, '//*[@class="pesquisaTituloEsquerda"]/a')
    list_tree = [element.text for element in tree_elements]
    trees = remove_items(list_tree, '')  # Remover elementos vazios

    abts = driver.find_elements(By.XPATH, '//*[@class="pesquisaSnippet"]')
    list_abts = [element.text for element in abts]
    
    # Adicionando a verificação de tamanho do abstract
    max_length = 500  # Defina o tamanho máximo permitido
    list_abts = [abt for abt in list_abts if len(abt) <= max_length]

    unidades = driver.find_elements(By.XPATH, '//*[@class="pesquisaMetatag"]')
    list_uni = [element.text.split(':') for element in unidades]
    info = [sublist_uni[1] for sublist_uni in list_uni if len(sublist_uni) > 1]  # Removendo listas vazias

    rows = driver.find_elements(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr')
    links = []
    files_name = []
    for i in range(1, len(rows)+1, 3):
        try:
            a = driver.find_element(By.XPATH, f'//*[@id="conteudo"]/table/tbody/tr[{i}]/td[2]/a')
            time.sleep(0.5)
            link = a.get_attribute('href')
            file_name=a.text
            links.append(link)
            files_name.append(file_name)
        except Exception as e:
            print(f"Erro ao processar a linha {i}: {e}")

    # print(files_name)

    # Inicializando o encurtador de links
    shortener = pyshorteners.Shortener(api_key='your_api_key', provider='isgd')
    links_curtos = []
    erros = []
    
    for link in links:
        try:
            # Separando a parte fixa e o hash do link
            base_url, hash_value = link.split('infra_hash=')
            
            # Verificando o tamanho da parte fixa do link antes de encurtar
            if len(base_url) > 1000:  # Ajuste o limite conforme necessário
                raise ValueError(f"Link base muito longo: {base_url}")
            
            # Encurtando a parte fixa do link
            link_curto = shortener.tinyurl.short(base_url)
            time.sleep(0.5)
            
            # Recriando o link com o hash
            link_com_hash = link_curto + 'infra_hash=' + hash_value
            
            links_curtos.append(link_com_hash)
            time.sleep(0.5)
        except Exception as e:
            erros.append((link, str(e)))
            links_curtos.append(None)
            
    dados = {
        "Número do Processo": trees[::2],
        "Documento": trees[1::2],
        "Resumo": list_abts,
        "Unidade": info[::3],
        "Usuário": info[1::3],
        "Data de Inclusão": info[2::3],
        "Links": links_curtos
    }
        
    df = pd.DataFrame(dados)
    
    original_window = driver.current_window_handle 
    
    for index, (link, name) in enumerate(zip(links, files_name)):
        driver.switch_to.new_window('tab')
        driver.get(link)
        with open(f'./colegiados_{name}.html', 'w', encoding='utf-8', errors='ignore') as file:
            file.write(driver.page_source)        
        driver.close()
        driver.switch_to.window(original_window)
    
    return df

col

col1 = extrair_dados1(driver)

In [27]:
def extrair_dados(driver):     
    """
    Função para realizar web scraping no SEI e retornar os dados em um DataFrame.

    Args:
        driver (webdriver): Instância do Selenium WebDriver.

    Returns:
        pd.DataFrame: DataFrame contendo os dados extraídos.
    """
    def remove_items(lista, item): 
        """Remove todos os itens iguais a `item` de uma lista.""" 
        return [i for i in lista if i != item]

    # Extraindo os elementos da pesquisa
    tree_elements = driver.find_elements("xpath", '//*[@class="protocoloNormal"]/a')
    list_tree = [element.text for element in tree_elements]
    trees = remove_items(list_tree, '')  # Remover elementos vazios

    unidades = driver.find_elements("xpath", '//*[@class="pesquisaMetatag"]')
    list_uni = [element.text.split(':') for element in unidades]
    info = [sublist_uni[1] for sublist_uni in list_uni if len(sublist_uni) > 1]  # Removendo listas vazias

    rows = driver.find_elements("xpath", '//*[@id="conteudo"]/table/tbody/tr')
    links = []
    
    for i in range(1, len(rows), 3):
        try:
            a = driver.find_element("xpath", f'//*[@id="conteudo"]/table/tbody/tr[{i}]/td[1]/a[2]')
            time.sleep(0.5)
            link = a.get_attribute('href')
            # print(link)
            time.sleep(0.5)
            links.append(link)
        except Exception as e:
            print(f"Erro ao processar a linha {i}: {e}")

    # Inicializando o encurtador de links
    shortener = pyshorteners.Shortener(api_key='your_api_key', provider='isgd')
    links_curtos = []
    erros = []
    
    for link in links:
        try:
            # Separando a parte fixa e o hash do link
            base_url, hash_value = link.split('infra_hash=')
            
            # Verificando o tamanho da parte fixa do link antes de encurtar
            if len(base_url) > 1000:  # Ajuste o limite conforme necessário
                raise ValueError(f"Link base muito longo: {base_url}")
            
            # Encurtando a parte fixa do link
            link_curto = shortener.tinyurl.short(base_url)
            time.sleep(0.5)
            
            # Recriando o link com o hash
            link_com_hash = link_curto + 'infra_hash=' + hash_value
            
            links_curtos.append(link_com_hash)
            time.sleep(0.5)
        except Exception as e:
            erros.append((link, str(e)))
            links_curtos.append(None)

    # Organizando os dados para o DataFrame
    dados = {
        "Número do Processo": trees[::2],
        "Documento": trees[1::2],
        # "Resumo": list_abts,  # Caso esteja disponível
        "Unidade": info[::3],
        "Usuário": info[1::3],
        "Data de Inclusão": info[2::3],
        "Links": links_curtos
    }

    df = pd.DataFrame(dados)
    return df

col = extrair_dados(driver)

//*[@id="conteudo"]/table/tbody/tr[1]/td[1]/a[2]
//*[@id="conteudo"]/table/tbody/tr[4]/td[1]/a[2]
//*[@id="conteudo"]/table/tbody/tr[7]/td[1]/a[2]

//*[@id="conteudo"]/div[2]/div[3]/a
//*[@id="conteudo"]/div[2]/div[2]/a[1]

next_page = driver.find_element("xpath", '//*[@id="conteudo"]/div[2]/div[3]/a')
print(next_page)

proxima_href = next_page.get_attribute('href')
print(proxima_href)

next_page.click()

dados_consolidados = pd.DataFrame()  # DataFrame vazio para acumular os dados

# Extrair os dados da página atual e adicionar ao DataFrame consolidado
df_pagina = extrair_dados(driver)
dados_consolidados = pd.concat([dados_consolidados, df_pagina], ignore_index=True)
# Procurar o botão "Próxima"
next_page = driver.find_element("xpath", '//*[@id="conteudo"]/div[2]/div[3]/a')
proxima_href = next_page.get_attribute('href')
print(proxima_href)
time.sleep(3)

In [6]:
def navegar_paginas(driver):
    """
    Loop para navegar por todas as páginas até que não haja mais um botão 'Próxima'.
    Retorna um DataFrame consolidado com os dados de todas as páginas.
    """
    dados_consolidados = pd.DataFrame()  # DataFrame vazio para acumular os dados

    while True:
        try:
            # Extrair os dados da página atual e adicionar ao DataFrame consolidado
            df_pagina = extrair_dados(driver)
            dados_consolidados = pd.concat([dados_consolidados, df_pagina], ignore_index=True)

            # Procurar o botão "Próxima"
            next_page = driver.find_element("xpath", '//*[@id="conteudo"]/div[2]/div[3]/a')

            # Verificar se o botão "Próxima" tem o atributo 'href'
            proxima_href = next_page.get_attribute('href')
            print(proxima_href)
            time.sleep(3)
            if not proxima_href:
                print("Não há mais páginas. Encerrando navegação.")
                break  # Sai do loop se não houver link para a próxima página

            # Clicar no botão "Próxima"
            next_page.click()
            time.sleep(10)  # Aguarda o carregamento da próxima página

        except NoSuchElementException:
            print("Botão 'Próxima' não encontrado. Encerrando navegação.")
            break  # Sai do loop se o botão "Próxima" não existir

        except Exception as e:
            print(f"Erro inesperado: {e}")
            break  # Sai do loop em caso de erro inesperado

    # Fechar o navegador após o término
    # driver.close()
    # driver.quit()

    return dados_consolidados

In [ ]:
dados = navegar_paginas(driver)

In [ ]:
def realizar_login(url, login1, password1, orgao1):
    """
    Função para realizar login no sistema SEI.

    Args:
        url (str): URL do sistema SEI.
        login1 (str): Nome de usuário para login.
        password1 (str): Senha para login.
        orgao1 (str): Nome do órgão para acesso.

    Returns:
        webdriver: Instância do WebDriver com o usuário autenticado.
    """
    try:
        # Inicializa o navegador
        driver = webdriver.Chrome()
        driver.implicitly_wait(0.5)
        driver.get(url)

        # Localiza os elementos de login
        login = driver.find_element(By.XPATH, '//*[@id="txtUsuario"]')
        password = driver.find_element(By.XPATH, '//*[@id="pwdSenha"]')
        orgao = driver.find_element(By.XPATH, '//*[@id="selOrgao"]')
        submit_button = driver.find_element(By.XPATH, '//*[@id="Acessar"]')

        # Preenche as credenciais
        login.send_keys(login1)
        password.send_keys(password1)
        orgao.send_keys(orgao1)

        # Realiza o login
        submit_button.click()
        time.sleep(3)  # Aguarda carregamento da página após o login

        print("Login realizado com sucesso!")
        return driver
    except Exception as e:
        print(f"Erro ao realizar o login: {e}")
        return None



# Exemplo de uso
url = 'https://sei.economia.gov.br/'
login1 = 'daiana.sales@gestao.gov.br'
password1 = "D!vertidamente081099"
orgao1 = "MGI"

# Realiza login
driver = realizar_login(url, login1, password1, orgao1)

# Se o login foi bem-sucedido, realiza a busca
if driver:
    buscar_arquivos(driver)

In [ ]:
print(dados)

In [ ]:
dados

In [ ]:
from io import BytesIO
def gerar_excel(df):
    output = BytesIO()
    with pd.ExcelWriter(output, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Doc_SEI")
    processed_data = output.getvalue()
    return processed_data

In [ ]:
excel_data = gerar_excel(dados)

In [ ]:
print(excel_data)